In [ ]:
!pip install tf-keras-vis

In [ ]:
## load package
%reload_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tf_keras_vis.utils import num_of_gpus
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from sklearn.model_selection import train_test_split
_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0 as Model

model = Model(weights='imagenet', include_top=True)
model.compile(
        loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
    )
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_path = "/content/drive/MyDrive/Subset_1-40_81-120/Image_subset_032224"
SCORECAM_path = "/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SCORECAM/"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split


In [ ]:
from PIL import Image

def change_size_for_model(image):
    image = Image.fromarray(image)
    resized_image = image.resize((224, 224))
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

def change_size_for_original(image, original_shape):
    image = Image.fromarray(image)
    resized_image = image.resize(original_shape) #resize image
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

In [ ]:
start_idx =0
end_idx = 41

In [ ]:
list_of_dir =list()
for i in range(start_idx,end_idx):
  list_of_dir.append(str(i))

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear, ModelModifier

replace2linear = ReplaceToLinear()
def model_modifier_function(cloned_model):
    cloned_model.layers[-1].activation = tf.keras.activations.linear

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.scorecam import Scorecam
# from tf_keras_vis.utils import num_of_gpus

In [ ]:
scorecam = Scorecam(model,
                    model_modifier=model_modifier_function,
                  clone=True)

In [ ]:
from matplotlib import cm

# for j in range(len(list_of_dir)):
for j in range(1):
  list_of_names = os.listdir(image_path+'/'+str(list_of_dir[j]))
  list_of_names.sort()
  original_image_array = list()

  for img_name in list_of_names:
    # Load the image
    image = Image.open(image_path+'/'+str(list_of_dir[j])+'/'+img_name)

    # Convert the image to a numpy array
    original_image_array.append(np.array(image))

  resized_image_array = list()
  for img in original_image_array:
    resized_image_array.append(change_size_for_model(img))

  resized_image_array = np.array(resized_image_array)

  ################
  # Your code

  result = model.predict(resized_image_array)
  saliency_map = list()
  for i in range(resized_image_array.shape[0]):
    X = resized_image_array[i]
    score = CategoricalScore([np.argmax(result[i])])
    print(np.argmax(result[i]))
    cam = scorecam(score, X, penultimate_layer=-1)
    heatmap = np.uint8(cm.jet(cam[0])* 255)
    plt.figure()
    plt.imshow(X, cmap='gray')
    plt.imshow(heatmap, cmap='jet', alpha=0.5)
    plt.show()
    saliency_map.append(cam)
    # plt.imshow(saliency_map[i].reshape(224,224), cmap='gray')
    # plt.show()
    # plt.pause(1)

  ################

  folder_for_exp = SCORECAM_path + str(list_of_dir[j])
  if not os.path.exists(folder_for_exp):
      # The folder does not exist, so create it
      os.makedirs(folder_for_exp, exist_ok=True)
      print(f"Folder '{folder_for_exp}' was created.")
  else:
      # The folder already exists
      print(f"Folder '{folder_for_exp}' already exists.")

  for i in range(len(saliency_map)):
    a = 255*saliency_map[i].reshape(224,224)
    X = resized_image_array[i]
    #print(a.shape)
    # plt.figure()
    # plt.hist(a.flatten(), bins=256, range=(0, 255))
    mod = np.percentile(a.flatten(),20)
    # print(mod)
    a[np.where(a >= mod)] = 255
    a[np.where(a < mod)] = 0

    plt.figure()
    plt.imshow(X, cmap='gray')
    # plt.imshow(heatmap, cmap='jet', alpha=0.5)

    plt.imshow(a, cmap='binary_r', alpha = 0.5)
    plt.show()
    plt.pause(1)
    plt.imsave(folder_for_exp+'/' +str(list_of_names[i]), a, cmap='binary_r')

In [ ]:
plt.imshow(255*saliency_map[0].reshape(224,224), cmap='gray')
plt.show()